In [ ]:
# import necassary libriries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load dataset
statistics = pd.read_csv('/Users/franziskadettmann/neue-fische-traum/datasets/statistics_20210707.csv')

In [ ]:
# Convert column `yearmonth` to datetime
statistics['yearmonth'] = pd.to_datetime(statistics['yearmonth'])

In [ ]:
# Heatmap
corr = statistics.corr()
ax = sns.heatmap(
    corr,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

In [ ]:
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
# Plot
plt.figsize
sns.displot(data=statistics, x='yearmonth', y='inquiry_count')